In [5]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import openai
import base64
import requests
from io import BytesIO
import json

# Global variable to store the conversation history
conversation_history = []

# Function to encode the image to base64
def encode_image(image_data):
    return base64.b64encode(image_data).decode('utf-8')

def send_message(b=None):
    global api_key, conversation_history, image_upload
    message = message_input.value
    temperature = temperature_control.value
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    
    # Construct the payload
    payload = {
        "model": "gpt-4-vision-preview",
        "messages": conversation_history,
        "max_tokens": 2000,
        "temperature": temperature
    }
    
    # Check if there is an image to send
    if image_upload.value:
        try:
            # Get the first uploaded file
            uploaded_file = next(iter(image_upload.value.values()))
            image_data = uploaded_file['content']
            base64_image = encode_image(image_data)
            user_message = {
                "role": "user",
                "content": [
                    {"type": "text", "text": message},
                    {"type": "image_url", "image_url": f"data:image/jpeg;base64,{base64_image}"}
                ]
            }
        except Exception as e:
            print(f"Error processing the uploaded file: {e}")
            return
    else:
        user_message = {
            "role": "user",
            "content": [{"type": "text", "text": message}]
        }
    
    # Append the user's message to the conversation_history
    conversation_history.append(user_message)
    
    # Display the user's text message
    messages_display.children += (widgets.Label(value=f"You: {message}"),)
    
    # Display the user's image if uploaded
    if image_upload.value:
        image = widgets.Image(
            value=image_data,
            format='png',
            width=300,
            height=400,
        )
        messages_display.children += (image,)
    
    # Send the message
    try:
        response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
        response.raise_for_status()
        response_data = response.json()
        if 'choices' in response_data and response_data['choices']:
            ai_message_content = response_data['choices'][0]['message']['content']
            ai_message = {
                "role": "assistant",
                "content": ai_message_content
            }
            # Append the AI's response to the conversation_history
            conversation_history.append(ai_message)
            messages_display.children += (widgets.Label(value=f"GPT-4V: {ai_message_content}"),)
        else:
            print("Failed to send message. Response did not contain expected data.")
    except Exception as err:
        print(f"An error occurred: {err}")
    finally:
        # Clear the input
        message_input.value = ""
        # Reset the image upload widget
        reset_image_upload()

# Function to reset the image upload widget
def reset_image_upload():
    global image_upload
    new_image_upload = widgets.FileUpload(description='Upload Image', accept='image/*', multiple=False)
    controls_box.children = list(controls_box.children[:-5]) + [new_image_upload] + list(controls_box.children[-4:])
    image_upload = new_image_upload

# Function to clear the conversation
def clear_conversation(b=None):
    global conversation_history
    conversation_history.clear()
    messages_display.children = []

# Function to download the conversation history as Markdown
def download_conversation(b=None):
    # Convert conversation_history to Markdown format
    conversation_md = ""
    for message in conversation_history:
        role = message['role']
        content = message['content']
        if role == 'user':
            # Extract the text from the user's message content
            if isinstance(content, list) and content and 'text' in content[0]:
                user_text = content[0]['text']
            else:
                user_text = str(content)  # Fallback to converting whatever content is to string
            conversation_md += f"user: {user_text}\n"
        elif role == 'assistant':
            # Assistant messages are assumed to be plain text
            conversation_md += f"assistant: {content}\n"

    b64 = base64.b64encode(conversation_md.encode()).decode()
    payload = f"data:text/markdown;base64,{b64}"
    downloader = widgets.HTML(
        value=f'<a download="conversation.md" href="{payload}" target="_blank">Download</a>'
    )
    display(downloader)

# Function to save the API key
def save_api_key(b):
    global api_key
    api_key = api_key_input.value
    with output:
        clear_output()
        print("API Key saved successfully.")

# API Key Configuration
api_key_input = widgets.Text(description='API Key:', placeholder='Enter your OpenAI API key here...')
api_key_button = widgets.Button(description="Save API Key")
api_key_button.on_click(save_api_key)

# Chat Interface
message_input = widgets.Textarea(description='Message:')
image_upload = widgets.FileUpload(description='Upload Image', accept='image/*', multiple=False)
send_button = widgets.Button(description='Send Message')
send_button.on_click(send_message)
clear_button = widgets.Button(description='Clear Conversation')
clear_button.on_click(clear_conversation)
download_button = widgets.Button(description='Download Conversation')
download_button.on_click(download_conversation)
messages_display = widgets.VBox([])
temperature_control = widgets.FloatSlider(value=0.7, min=0, max=1.0, step=0.01, description='Temperature:')

# Display and Interaction
output = widgets.Output()
api_key_box = widgets.VBox([api_key_input, api_key_button])
controls_box = widgets.VBox([message_input, image_upload, send_button, clear_button, download_button, temperature_control])
chat_box = widgets.VBox([controls_box, messages_display])

# Layout the widgets
display(api_key_box, chat_box, output)

# Initialize the image upload widget
reset_image_upload()


Output()

HTML(value='<a download="conversation.md" href="data:text/markdown;base64,dXNlcjogaGVsbG8KYXNzaXN0YW50OiBIZWxs…